<a href="https://colab.research.google.com/github/rahiakela/natural-language-processing-research-and-practice/blob/main/applied-text-analysis-with-python/4_text_vectorization_and_transformation_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Text Vectorization and Transformation Pipeline

In this notebook, we will demonstrate how to use the vectorization process to combine
linguistic techniques from NLTK with machine learning techniques in Scikit-Learn
and Gensim, creating custom transformers that can be used inside repeatable and
reusable pipelines.

In order
to perform machine learning on text, we need to transform our documents into vector
representations such that we can apply numeric machine learning. This process is
called feature extraction or more simply, vectorization, and is an essential first step
toward language-aware analysis.

Representing documents numerically gives us the ability to perform meaningful ana
lytics
and also creates the instances on which machine learning algorithms operate

For this reason, we must now make a critical shift in how we think about language—
from a sequence of words to points that occupy a high-dimensional semantic space.
Points in space can be close together or far apart, tightly clustered or evenly distributed.

By
encoding similarity as distance, we can begin to derive the primary components of
documents and draw decision boundaries in our semantic space.

The simplest encoding of semantic space is the bag-of-words model, whose primary
insight is that meaning and similarity are encoded in vocabulary.

##Setup

In [22]:
import nltk
import string
import numpy as np

from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Binarizer
import gensim

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
corpus = [
  "The elephant sneezed at the sight of potatoes.",
  "Bats can see via echolocation. See the bat sight sneeze!",
  "Wondering, she opened the door to the studio.",
]

##Words in Space

We will look at four types of vector encoding—frequency,
one-hot, TF–IDF, and distributed representations—and discuss their implementations
in Scikit-Learn, Gensim, and NLTK.

To set this up, let’s create a list of our documents and tokenize them for the proceeding
vectorization examples.

In [4]:
def tokenize(text):
  stem = nltk.stem.SnowballStemmer("english")
  text = text.lower()

  for token in nltk.word_tokenize(text):
    if token in string.punctuation:
      continue
    yield stem.stem(token)

###Frequency Vectors

In [5]:
# With NLTK
def vectorize(doc):
  features = defaultdict(int)
  for token in tokenize(doc):
    features[token] += 1
  return features

vectors = map(vectorize, corpus)
for vector in vectors:
  print(dict(vector))

{'the': 2, 'eleph': 1, 'sneez': 1, 'at': 1, 'sight': 1, 'of': 1, 'potato': 1}
{'bat': 2, 'can': 1, 'see': 2, 'via': 1, 'echoloc': 1, 'the': 1, 'sight': 1, 'sneez': 1}
{'wonder': 1, 'she': 1, 'open': 1, 'the': 2, 'door': 1, 'to': 1, 'studio': 1}


In [6]:
# In Scikit-Learn
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(corpus)
for vector in vectors:
  print(vector)

  (0, 16)	2
  (0, 6)	1
  (0, 14)	1
  (0, 0)	1
  (0, 12)	1
  (0, 7)	1
  (0, 9)	1
  (0, 16)	1
  (0, 12)	1
  (0, 2)	1
  (0, 3)	1
  (0, 10)	2
  (0, 18)	1
  (0, 5)	1
  (0, 1)	1
  (0, 13)	1
  (0, 16)	2
  (0, 19)	1
  (0, 11)	1
  (0, 8)	1
  (0, 4)	1
  (0, 17)	1
  (0, 15)	1


In [14]:
# The Gensim way
tokenized_corpus = [list(tokenize(doc)) for doc in corpus]
id2word = gensim.corpora.Dictionary(tokenized_corpus)
vectors = [id2word.doc2bow(doc) for doc in tokenized_corpus]
vectors

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2)],
 [(4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 2), (11, 1)],
 [(6, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)]]

###One-Hot Encoding

In [15]:
# With NLTK
def vectorize(doc):
  return {token: True for token in tokenize(doc)}

vectors = map(vectorize, corpus)

for vector in vectors:
  print(vector)

{'the': True, 'eleph': True, 'sneez': True, 'at': True, 'sight': True, 'of': True, 'potato': True}
{'bat': True, 'can': True, 'see': True, 'via': True, 'echoloc': True, 'the': True, 'sight': True, 'sneez': True}
{'wonder': True, 'she': True, 'open': True, 'the': True, 'door': True, 'to': True, 'studio': True}


In [18]:
# In Scikit-Learn
freq = CountVectorizer()
tokenized_corpus = freq.fit_transform(corpus)

print(len(tokenized_corpus.toarray()[0]))

one_hot = Binarizer()
vectors = one_hot.fit_transform(tokenized_corpus.toarray())
for vector in vectors:
  print(vector)

20
[1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0]
[0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0]
[0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 0 1]


In [21]:
# In Scikit-Learn alternate way
freq = CountVectorizer(binary=True)
tokenized_corpus = freq.fit_transform(corpus)

print(len(tokenized_corpus.toarray()[0]))

for vector in tokenized_corpus.toarray():
  print(vector)

20
[1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0]
[0 1 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0]
[0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 0 1]


In [27]:
# The Gensim way
tokenized_corpus = [list(tokenize(doc)) for doc in corpus]
id2word = gensim.corpora.Dictionary(tokenized_corpus)

vectors = np.array([[(token[0], 1) for token in id2word.doc2bow(doc)] for doc in tokenized_corpus])

for vector in vectors:
  print(vector)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]
[(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]
[(6, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


###TF-IDF